<a href="https://colab.research.google.com/github/matthewlai12/ECE-Lab/blob/main/ECE449Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ECE 449 Lab 2: Matthew Lai


##Section 1: Tree of Thought Agent
In this section, I will discuss general Tree-of-Thought prompting.

Tree-of-Thought prompting revolves around the idea of breaking down a thought process into steps. This forms a "Tree-of-Thought," and as the name suggests, we want to decompose a problem into parts that the model can better understand. The model will respond with thoughts based on initial prompts or context.

When a model is given context or a prompt, its goal is to decompose and understand the context we are in. Once the model can digest this information, it can apply a search algorithm such as Depth-First Search in our case, and explore our possible solutions.

As the model continues to work on the problem it repeats the process. Understand the current context, generate a thought based on this context, and continue exploring possible solutions until the end is met.

## LLM Prompt Engineering Lab
### Tree of Thought Prompting with GPT4o-mini

Preliminaries: import libraries, read in the OpenAI API key and pass it to an OpenAI object constructor

In [1]:
!pip install python-dotenv anytree openai
!git clone https://github.com/rfwarn/ASCII_LLM_Maze.git

fatal: destination path 'ASCII_LLM_Maze' already exists and is not an empty directory.


In [2]:
# Import libraries, set API key
from anytree import Node, RenderTree

# Added the following lines in order to import the maze,
# as per eclass instructions.
import sys
sys.path.insert(1,'/content/ASCII_LLM_Maze')

import ASCIImaze
import os
import re

# This chunk of code reads an OpenAI API key from a chosen environment variable, which must be predefined.
# For my own dev, I'm using my personal API key. This will not be provided to others, you must use one you have
# access to.
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

from openai import OpenAI

client = OpenAI(
    api_key=os.environ['OPENAI_D31_09191141_KEY']
)


<b>First prompt:</b> Recognize the ASCII maze, and format it as a single-line string, with the different lines of the maze delimited by "|"

In [3]:
extract_maze_system_prompt = """
  You are to extract and format a maze represented in ASCII. The maze consists of multiple lines representing rows of the maze.

  Your job is to format the entire maze as a single-line string, where each row of the maze is separated by the '|' character.

  In this maze:
  - '#' represents a wall
  - '*' represents your current position
  - 'S' represents the start of the maze
  - 'E' represents the end of the maze

  IMPORTANT NOTES:
  - Only return the maze formatted as a single line with rows delimited by '|'
  - Do not return any explanations, text, or characters
  - The maze's characters should remain unchanged except for the row delimiter

  Provided next are some example mazes:

  Example 1:
  If the maze looks like this:
  #######
  # S####
  #  ####
  ## ####
  ## ####
  ##E####
  #######

  Return this:
  #######|# S####|#  ####|## ####|## ####|##E####|#######

  Example 2:
  If the maze looks like this:
  #######
  #S#####
  #  ####
  ## ####
  ##   ##
  ####E##
  #######

  Return this:
  #######|#S#####|#  ####|## ####|##   ##|####E##|#######

  Now, format this maze by formatting it in a single string and return the maze
  {maze_ascii}
"""

<b>Second prompt:</b> Format the 3x3 neighborhood in the same manner as the ASCII maze.

In [4]:
generate_thought_context_system_prompt = """
  You are now solving the ASCII maze, you need to extract the 3x3 neighbourhood that you are currently in.

  Your job is to format the 3x3 neighbourhood that you are in a single-line string, where each row of the 3x3 neighbourhood is separated by the '|' character.

  IMPORTANT NOTES:
  - Only return the 3x3 neighbourhood formatted as a single line with rows delimited by '|'
  - Do not return any explanations, text, or characters
  - The 3x3 neighbourhood's characters should remain unchanged except for the row delimiter

  Provided next are some example 3x3 neighbourhood:

  Example 1:
  If the 3x3 neighbourhood looks like this:
  # #
  #*
  # #

  Return this:
  # #|#* |# #

  Example 2:
  If the 3x3 neighbourhood looks like this:
  # #
  #*
  ###

  Return this:
  # #|#* |###


  Now, format this 3x3 neighborhood as a single string and return it:
  {neighborhood_ascii}
"""

<b>Third prompt:</b> Based on the refomatted ASCII maze and neighborhood from prompts 1 and 2 above, choose the next move to make.

In [5]:
generate_thought_system_prompt = """
  Now output the next move. Return this value as the capital letter from {vmove}.
  Do not output {backmove} unless it is the only move in {vmove}.
  Only return 1 move choice.

  Provided are example move choices:

  Example 1:
  Neighborhood: ###|E*#|###
  Valid moves: 'L' (Left)
  Backmove: None
  Choice: 'L'

  Example 2:
  Neighborhood: # #| *#|###
  Valid moves: 'L' (Left), 'U' (Up)
  Backmove: None
  Choice: 'L'

  Example 3:
  Neighborhood: ###|#* |##
  Valid moves: 'R'
  Backmove: 'R' (Right)
  Choice: 'R'
"""

In [6]:
# Possible models:
# gpt-3.5-turbo
# gpt-4o-mini

# Initialize global tracking variables
totalInTokens = 0
totalOutTokens = 0

# get_completion is a helper method suggested in the DeepLearning.ai course on prompt engineering.
# I modified this version to use the OpenAI 1.0 implementation. The "client" object was passed an API key in the first code box.
def get_completion(system_prompt, user_prompt, my_model="gpt-4o"):

    # We need to access our global variables
    global totalInTokens
    global totalOutTokens

    completion = client.chat.completions.create(
        model=my_model,
        messages=[
            {"role": "system",
             "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # We want to get the usage from the API call for the response\
    # Collecting usage tokens
    usage = completion.usage
    inTokens = usage.prompt_tokens
    outTokens = usage.completion_tokens

    # Incrementing usage tokens
    totalInTokens += inTokens
    totalOutTokens += outTokens

    # Print our results
    print("Total Input Tokens used this call: " + str(inTokens) + " Total Input Tokens used overall: " + str(totalInTokens))
    print("Total Output Tokens used this call: " + str(outTokens) + " Total Output Tokens used overall: " + str(totalOutTokens))

    return completion.choices[0].message


# -------------------------------------------------------------------------------------------------------------

# Utility function for displaying trees. Core code from the anytree package documentation.
def render_that_tree(mytree):
    for pre, _, node in RenderTree(mytree):
        print("%s%s" % (pre, node.name))

# ---------------------------------------------------------------------------------------------------------------

# Backtracking subroutine for Tree of Thought. Updates teh parent's state info to prune the dead-end branch we are
#  backtracking from (by setting that position in the stored 3x3 to '#').

def tree_backtrack(deadend_node, curr_move):

    curr_node = deadend_node.parent

    if deadend_node.move == 'D':
        temp = list(curr_node.name)
        temp[9] = '#'
        curr_node.name = ''.join(temp)

    elif deadend_node.move == 'U':
        temp = list(curr_node.name)
        temp[1] = '#'
        curr_node.name = ''.join(temp)

    elif deadend_node.move == 'L':
        temp = list(curr_node.name)
        temp[4] = '#'
        curr_node.name = ''.join(temp)

    elif deadend_node.move == 'R':
        temp = list(curr_node.name)
        temp[6] = '#'
        curr_node.name = ''.join(temp)

    else:
        print("Illegal move in backtracking")
        return None

    return curr_node

# ----------------------------------------------------------------------------------------------------------------

# Add node subroutine for Tree of Thought. Pretty short, but putting it here makes teh main loop cleaner.
#  This also pushes the tree updates out to subroutines, which is common and familiar. Leaves just teh LLM and game
#  logic in the main loop, which I think is good.
def add_node(curr_node, LLM_move_cleaned):
    # New node is a child of the current node. State has to be determined on next iteration. The move to reach this
    #  node from its parent is recorded as the "move" attribute. The move to reach its parent from this node
    #  is recorded as the "backtrack" attribute.

    # Have to determine the backtrack attribute by reversing LLM_cleaned_move. Another elif tree here.
    if LLM_move_cleaned.find('D') >= 0:
        rev = 'U'

    elif LLM_move_cleaned.find('U') >= 0:
        rev = 'D'

    elif LLM_move_cleaned.find('L') >= 0:
        rev = 'R'

    elif LLM_move_cleaned.find('R') >= 0:
        rev = 'L'

    else:
        print("Not a legal move")
        return None

    temp = Node(parent=curr_node, name="", move=LLM_move_cleaned, backtrack=rev)

    return temp


# ----------------------------------------------------------------------------------------------------------

def evaluate_thought(curr_local):
    # Pre: curr_local is the 3x3 neighborhood for the current location in the maze.
    # Format ###|#*#|###
    # Post: none
    # Returns: List of valid moves, each a 1-character string from {U, D, L, R}

    # Create output list, convert input string to a list.

    # Debug
    print(f"evaluate thought recevied {curr_local}")
    # Debug

    valid_moves = []
    maze_neigh = list(curr_local)
    # print(f"Local neighborhood string is {curr_local}")
    # print(f"Converted to a List it is {maze_neigh}")

    if maze_neigh[1] == ' ' or maze_neigh[1] == 'S' or maze_neigh[1] == 'E':
        valid_moves.append("U")
        # print("found U")

    if maze_neigh[4] == ' ' or maze_neigh[4] == 'S' or maze_neigh[4] == 'E':
        valid_moves.append("L")
        # print("found L")

    if maze_neigh[6] == ' ' or maze_neigh[6] == 'S' or maze_neigh[6] == 'E':
        valid_moves.append("R")
        # print("found R")

    if maze_neigh[9] == ' ' or maze_neigh[9] == 'S' or maze_neigh[9] == 'E':
        valid_moves.append("D")
        # print("found D")

    return valid_moves

# ----------------------------------------------------------------------------------------------------------

# Guards for 1st iteration, context request.
ready_for_input = False
need_context_update = True
mazeTreeStart = Node(name="", move="", backtrack="")
curr_node = mazeTreeStart
# Debug
counter = 0

# The maze game loop is contained in the maze.solve() method. Call the constructor, and then enter a for loop
#  to receive outputs from maze.solve(), and respond with our inputs.
maze = ASCIImaze.Maze("maze_map1", show_moves=False, show_coords=False)
for output in maze.solve():
    if (not ready_for_input):
        # On the first iteration, we get the maze to solve. Bring it into the LLM, then goto next iteration; that
        #  will be the first opportunity to pass an input to the ASCIImaze solver.
        # print(output)
        # Going to pull the maze into the LLM's chat history now.
        user_prompt=f"""```{output}```"""
        maze_to_solve = get_completion(extract_maze_system_prompt, user_prompt)
        #print(maze_to_solve.content)
        ready_for_input = True
        continue

    if (need_context_update):
        # Grab context by passing '3' to ASCIIMaze. Isolated so can use stored context instead.
        # Must go to next iteration b/c only then will 'output' be updated with the 3x3 neighborhood.
        maze.get_user_move('3')
        need_context_update = False
        continue

    if "Maze solved!" in output:
        # Victory!
        print(output)
        break


    # Pull in the current 3x3 neighborhood, IF the current node does not already have a state. Otherwise, this is a
    #  backtracking situation.

    # Debug
    print(f"Output reads: {output}")
    if curr_node.name == "":
        # Case for a node not previously visited.
        user_prompt = f"```{output}```"
        neigh = get_completion(generate_thought_context_system_prompt, user_prompt)
        curr_node.name = neigh.content


    # Evaluate the current position; this is actually evaluating the through generated in the previous round.
    # Possible results are {{valid moves}, <just the backtrack direction>}
    # Backtrack if so indicated, generate a new thought if there are other valid moves.
    # The ASCIImaze.maze.solve() method, the main game loop, outputs a message when you correctly solve the maze, you
    #  don't need to figure out if the LLM has won the game. Just check for the victory message. Done above.


    print(f"Maze neighborhood is {curr_node.name}")

    valid_moves_cleaned = evaluate_thought(curr_node.name)


    print(f"Valid moves are {valid_moves_cleaned}")



    if len(valid_moves_cleaned) == 1 and valid_moves_cleaned[0] == curr_node.backtrack:
        # The only move detected is the backtrack direction, so follow it, both in the maze and the tree.
        maze.get_user_move(curr_node.backtrack)
        curr_node = tree_backtrack(curr_node, curr_node.backtrack)

        # Do not update context, the state in curr_node has already been updated. BUT this blows up an assumption :(
        need_context_update = False

        # The valid moves from the last eval step are no longer correct. Go to the next iteration, get a new move set.
        continue

    # Generate a thought
    # The LLM chooses a next move, which will be a child of the current node
    # Again, inner monologues are used to recreate the 2D mazes.
    user_prompt = f"""```{maze_to_solve.content}, {curr_node.name}```"""

    # Adding the backtrack move to thought generation. Remember to deal with root node (no parent).
    valid_moves_temp = ", ".join(valid_moves_cleaned)

    # Initial value for LLM_move, for loop entry
    LLM_move_cleaned = ['Z']

    # Loop to check for valid move.
    # LLM_move_cleaned should be a single-element list. If that element is not part of the valid move set, try again.
    while valid_moves_temp.find(LLM_move_cleaned[0]) < 0:

        print(f"Moves going to LLM are {valid_moves_temp}")

        if curr_node.parent != None:
            LLM_move = get_completion(generate_thought_system_prompt.format(vmove=valid_moves_temp,
                                                                    backmove=curr_node.backtrack), user_prompt)
        else:
            LLM_move = get_completion(generate_thought_system_prompt.format(vmove=valid_moves_temp,
                                                                        backmove=None), user_prompt)

        LLM_move_temp = re.sub('\&\&\&.+\&\&\&', '', LLM_move.content, flags=re.DOTALL)
        # Again, another regex needed.
        LLM_move_cleaned = re.findall("[A-Z]", LLM_move_temp)
        print(f"I choose move {LLM_move_cleaned[0]}")
    # Error check
    if len(LLM_move_cleaned) > 1:
        print("Extra stuff in the generated thought, aborting")
        break


    # Make the chosen move, and add the corresponding node to the tree.
    maze.get_user_move(LLM_move_cleaned[0])
    curr_node = add_node(curr_node, LLM_move_cleaned[0])
    if curr_node is None:
        print("You goofed.")
        print(LLM_move_cleaned)
    # Debug
    counter = counter + 1
    print(f"I choose move {LLM_move_cleaned}")
    print(f"This is move {counter}")

    # Need the next 3x3 neighborhood.
    need_context_update = True

print("I finished")



Total Input Tokens used this call: 489 Total Input Tokens used overall: 489
Total Output Tokens used this call: 42 Total Output Tokens used overall: 42
Output reads: Here is a 3x3 of your current position:
###
#* 
# #
Enter a move direction (U, D, L, R, or 3 (for a 3x3 of the current position)): 
Total Input Tokens used this call: 296 Total Input Tokens used overall: 785
Total Output Tokens used this call: 7 Total Output Tokens used overall: 49
Maze neighborhood is ###|#* |# #
evaluate thought recevied ###|#* |# #
Valid moves are ['R', 'D']
Moves going to LLM are R, D
Total Input Tokens used this call: 230 Total Input Tokens used overall: 1015
Total Output Tokens used this call: 1 Total Output Tokens used overall: 50
I choose move R
I choose move ['R']
This is move 1
Output reads: Here is a 3x3 of your current position:
###
S* 
 ##
Enter a move direction (U, D, L, R, or 3 (for a 3x3 of the current position)): 
Total Input Tokens used this call: 295 Total Input Tokens used overall: 1310

##Section 2: LLM Performance with Tree-of-Thought Prompting

** IMPORTANT NOTE: model gpt-4o was used as the mini was at the limit.

In this section we will discuss 3 key points when evaluating whether or not LLM's perform well when tasked with solving maze problems. Section 2 will be split into 3 sections: Performance of LLM with Tree-of-Thought prompting relative to this lab, costs of performing the task, and lastly an engineered opinion on whether or not LLMs should be tasked with solving mazes.

###Section 2.1: LLM Performance Relative to our Lab
In our lab, we gave the model 3 prompts. The first two prompts helped give the model the context it needed in order to make a decision. We used a multistep approach that made 2 API calls in order to make 1 decision. As mentioned in the description of Tree-of-Thought prompting, we need to give the model context based on each decision. This is why we used 2 steps. The first step is to get the current context of the maze, and the second is to advance our Depth-First Search algorithm to solve the maze. Prompt 1 focuses on giving the model the context for the entire maze, allowing the model to see where their current position is relative to the start and endpoint. This was useful for when the model needed to complete backtracking. Prompt 2 gives the model the current 3x3 neighbourhood that it is currently in, with the model at the centre. In this case, the prompt is important for the model to understand all of its current options. Prompt 3 gives the model the choice based on the last 2 prompts to choose a decision and a direction to advance the model through the searching algorithm.